# Get the Data 

In [2]:
import pandas as pd 
data = pd.read_csv("../datasets/raw_data.csv")
data.head()  

,Unnamed: 0,optionId,bill
0,0,751839,1
1,1,751782,1
2,2,745321,1
3,3,666652,1
4,4,751873,2


In [3]:
data.columns = ['index', 'option', 'bill']

In [4]:
data.head() 

,index,option,bill
0,0,751839,1
1,1,751782,1
2,2,745321,1
3,3,666652,1
4,4,751873,2


In [5]:
sample_set = data[:100]

In [6]:
sample_set.shape

(100, 3)

In [7]:
sample_set.bill.unique() 

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
      dtype=int64)

In [8]:
# create two list of options 
all_options = sample_set.option.unique()
all_options.shape

(85,)

In [9]:
import numpy as np 

half = int(np.floor(all_options.shape[0] / 2)) 
half 

42

In [10]:
left_options = all_options[:half] 
right_options = all_options[half:]

left_options.shape[0] + right_options.shape[0]

85

In [11]:
right_options[:10], right_options[:10]

(array([705527, 704329, 705522, 705502, 714910, 715013, 753517, 705521,
        677956, 750116], dtype=int64),
 array([705527, 704329, 705522, 705502, 714910, 715013, 753517, 705521,
        677956, 750116], dtype=int64))

In [12]:
import itertools
p = itertools.product(left_options, right_options)

In [13]:
option_pairs = np.array([v for v in p])

In [14]:
option_pairs[:3]

array([[751839, 705527],
       [751839, 704329],
       [751839, 705522]], dtype=int64)

In [15]:
option_rating = dict() 
bills = sample_set.bill.unique() 

bills[:3]

array([1, 2, 3], dtype=int64)

In [16]:
for bill in bills: 
    a = set(sample_set[sample_set.bill == bill]['option'])
    b = set([756488, 752059])
    print(a, b, a & b, bool(a & b))
    break 

{745321, 666652, 751782, 751839} {756488, 752059} set() False


In [17]:
set(option_pairs[0])

{705527, 751839}

In [18]:
%%time 
# traverse for one pair in all bills 
for pair in option_pairs: 
    pair_s = set(pair) 
    for bill in bills: 
        options = set(sample_set[sample_set.bill == bill]['option'])
        if(bool(pair_s & options)):
            option_rating[tuple(pair)] = option_rating.get(tuple(pair), 0) + 1  

CPU times: total: 16.3 s
Wall time: 16.7 s


In [20]:
import sys 

def get_size_KB(input): 
    kB = np.power(2, 20) 
    return sys.getsizeof(input) / kB  

In [21]:
def create_rating_matrix(dataset): 
    all_options = dataset.option.unique()
    half = int(np.floor(all_options.shape[0] / 2)) 
    left_options = all_options[:half] 
    right_options = all_options[half:]
    p = itertools.product(left_options, right_options)
    print(len(p))
    option_rating = dict() 
    bills = dataset.bill.unique() 

    k = 0 
    # traverse for one pair in all bills 
    for pair in p: 
        pair_s = set(pair) 
        print(f"Size of the result {get_size_KB(option_rating)}")
        if k == 1: 
            break
        for bill in bills: 
            options = set(dataset[dataset.bill == bill]['option'])
            if(bool(pair_s & options)):
                option_rating[tuple(pair)] = option_rating.get(tuple(pair), 0) + 1  
        
        k = k + 1 
        # bıdı bıdı 
    return option_rating 

In [22]:
len(data.option.unique())

9565

In [23]:
(37253 * 8) / np.power(2, 20)

0.28421783447265625

In [24]:
data_ordered = data.sort_values(by="option", ignore_index=True)

In [25]:
# data_ordered.to_csv("../datasets/ordered_by_options.csv")

In [26]:
data_ordered.head(5)

,index,option,bill
0,900771,10494,287600
1,748625,14310,239234
2,960707,16329,306550
3,900773,16716,287600
4,860419,16716,274508


# Binary Search Algorithm  


In [27]:
from bisect import bisect_left
 
def BinarySearch(a, x):
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    else:
        return -1

In [28]:
BinarySearch(data_ordered.option.to_numpy(), 2284)

-1

Find the bills for single one option 

In [29]:
ds = data_ordered.option.to_numpy() 
bs = data_ordered.bill.to_numpy() 
ds[:20], bs[:20 ]

(array([10494, 14310, 16329, 16716, 16716, 18511, 19216, 20198, 23880,
        23880, 31567, 32753, 32753, 32753, 41167, 41167, 41167, 41167,
        41167, 41580], dtype=int64),
 array([287600, 239234, 306550, 287600, 274508, 313566, 242116, 242126,
        293521, 287600, 287600, 264667, 258525, 264747, 266871, 325450,
        258709, 261779, 268516, 301226], dtype=int64))

In [30]:
ex_option = 2284 
idx_start = BinarySearch(ds, ex_option) 

bills = [bs[i] for i in range(idx_start, len(bs)) if ex_option == ds[i]]

bills

[]

In [31]:
s1 = set(np.concatenate((bills, [43,56,78,98,3456,675,34,767])))
s2 = set([43,56,99,7,8,89,34,45])

len(s1.intersection(s2))

3

# Rating Matrix with Binary Search 

In [32]:
def get_bills(options:np.array, option_pair:tuple, bills:np.array):
    idx_start = BinarySearch(options, option_pair[0]) 
    s1 = set([bills[i] for i in range(idx_start, len(bills)) if option_pair[0] == options[i]]) 
    idx_start = BinarySearch(options, option_pair[1]) 
    s2 = set([bills[i] for i in range(idx_start, len(bills)) if option_pair[1] == options[i]]) 
    return s1.intersection(s2)

In [33]:
def create_rating_matrix_BS(dataset_ordered, dataset_raw):
    all_options = dataset_raw.option.unique()
    half = int(np.floor(all_options.shape[0] / 2)) 
    left_options = all_options[:half] 
    right_options = all_options[half:]

    p = itertools.product(left_options, right_options)
    options = dataset_ordered.option.to_numpy() 
    bills = dataset_ordered.bill.to_numpy()
    result = dict()  
    
    i = 0 
    j = 0 
    for pair in p: 
        intersection = len(get_bills(options, pair, bills))
        if intersection > 0:
            result[tuple(pair)] = result.get(tuple(pair), 0) + intersection 
            
            i = i + 1
            if(i % 100 == 0):
                print("SIZE :", get_size_KB(result)) 
            
        j = j + 1 
        if(j % 1000 == 0):  
            break 
        
        

    return result 


In [34]:
# create_rating_matrix_BS(data_ordered, data)

In [35]:
data_sorted = data.sort_values(by=["bill", "option"]).drop("index", axis=1)
data_sorted.reset_index(drop=True, inplace=True)

data_sorted.head() 

,option,bill
0,666652,1
1,745321,1
2,751782,1
3,751839,1
4,751873,2


In [37]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

s = set([1, 2, 3, 4])

findsubsets(s, 2)

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

In [56]:
comibined_option = {}
bill_options = set() 
bill = sample_set.loc[0].bill 
for index, row in sample_set.iterrows():                # O(n)
    if(row.bill == bill): 
        bill_options.add(row.option) 
    else:
        if(len(bill_options) > 1):                      # len(bill_options) = O(1)
            subsets = findsubsets(bill_options, 2)      # O(?)
            for subset in subsets: 
                comibined_option[subset] = comibined_option.get(subset, 0) + 1 # avverage complexity O(1) 
        
        bill_options.clear() 
        bill_options.add(row.option)
        bill = row.bill       

In [39]:
findsubsets([1,2], 2)

[(1, 2)]

# Rating Matrix With Only One Traversal Approach 

In [70]:
def create_rating_matrix_OS(dataset):  
    """ 
    Input 
        dataset - includes bills and options only. It has to be ordered by bill, option. This is the requirement. 

    Output 
        combined options with ratings. 
    """
    comibined_option = {}
    bill_options = set() 
    bill = dataset.loc[0].bill 
    
    i = 0 
    for index, row in dataset.iterrows():                # O(n)
        if(row.bill == bill): 
            bill_options.add(row.option)
        else:
            if(len(bill_options) > 1):                      # len(bill_options) = O(1)
                subsets = findsubsets(bill_options, 2)      # O(?)
                for subset in subsets: 
                    comibined_option[subset] = comibined_option.get(subset, 0) + 1 # avverage complexity O(1)
                    
                    i = i + 1 
                    if(i % 100 == 0):
                        print(f"{i} th iteration for building. Size:", get_size_KB(comibined_option)) 

            bill_options.clear() 
            bill_options.add(row.option)
            bill = row.bill

    return comibined_option  

In [73]:
rating_matrix = create_rating_matrix_OS(data_sorted)

100 th iteration for building. Size: 0.00447845458984375
200 th iteration for building. Size: 0.008880615234375
300 th iteration for building. Size: 0.008880615234375
400 th iteration for building. Size: 0.01766204833984375
500 th iteration for building. Size: 0.01766204833984375
600 th iteration for building. Size: 0.01766204833984375
700 th iteration for building. Size: 0.01766204833984375
800 th iteration for building. Size: 0.035247802734375
900 th iteration for building. Size: 0.035247802734375
1000 th iteration for building. Size: 0.035247802734375
1100 th iteration for building. Size: 0.035247802734375
1200 th iteration for building. Size: 0.035247802734375
1300 th iteration for building. Size: 0.035247802734375
1400 th iteration for building. Size: 0.035247802734375
1500 th iteration for building. Size: 0.07039642333984375
1600 th iteration for building. Size: 0.07039642333984375
1700 th iteration for building. Size: 0.07039642333984375
1800 th iteration for building. Size: 0.0

KeyboardInterrupt: 